# Wakeword Training - Colab & JupyterLab Uyumlu

CUDA/GPU doğrulamalı, her 10 epoch checkpoint kaydeden, TorchMetrics ve TensorBoard entegrasyonlu eğitim defteri. Colab'da Drive'dan 7z ve npy içeriği kullanımı dahildir.

## 1) Plan ve TODO Listesi (Sequential Thinking)
Aşamalar: 1) ortam/paketler, 2) GPU doğrulama, 3) Colab-Drive montajı ve veri çıkarımı, 4) yol yapılandırması, 5) veri yükleyiciler, 6) model, 7) kayıp/optimizer/metrics, 8) TensorBoard, 9) eğitim (10 epoch checkpoint), 10) doğrulama, 11) checkpoint yükleme, 12) Colab TensorBoard, 13) GPU izleme.

In [ ]:
# Planı kod olarak yazdır
plan = [
    "Ortam ve paket kurulumları",
    "GPU doğrulama",
    "Colab-Drive montajı ve veri çıkarımı (7z + npy)",
    "Yol yapılandırması",
    "Veri yükleyiciler",
    "Model",
    "Kayıp/Optimizer/Metrics (TorchMetrics)",
    "TensorBoard",
    "Eğitim (10 epoch'ta bir checkpoint)",
    "Doğrulama/Test",
    "Checkpoint yükleme ve devam",
    "Colab TensorBoard",
    "GPU izleme"
]
for i, item in enumerate(plan, 1):
    print(f"{i}. {item}")

## 2) Paket Kurulumu ve Sürüm Sabitleme (JupyterLab ve Colab)

In [ ]:
# Colab ve lokal kurulum
import sys, os, platform
IN_COLAB = 'COLAB_GPU' in os.environ or 'google.colab' in sys.modules

# Temel paketler
!pip -q install --upgrade pip
!pip -q install torchmetrics>=1.4.0 tensorboard>=2.14.0 py7zr>=0.20.7 tqdm numpy<2.0

# Colab'da gerekiyorsa CUDA destekli PyTorch kurulumu (çoğu zaman hazır gelir)
if IN_COLAB:
    try:
        import torch
        print('Colab Torch sürümü:', torch.__version__)
    except Exception:
        # Örnek: CUDA 12.1 için
        !pip -q install --index-url https://download.pytorch.org/whl/cu121 torch torchvision torchaudio
        import torch
        print('Kurulan Torch sürümü:', torch.__version__)


## 3) CUDA/GPU Doğrulama ve İzleme

In [ ]:
# CUDA doğrulama ve nvidia-smi
import subprocess, shlex, time, threading
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))

# Basit GPU monitor iş parçacığı
gpu_monitor_stop = False

def gpu_monitor(interval=10):
    global gpu_monitor_stop
    while not gpu_monitor_stop:
        try:
            cmd = 'nvidia-smi --query-gpu=timestamp,index,name,utilization.gpu,utilization.memory,memory.used,memory.total,temperature.gpu --format=csv,noheader,nounits'
            out = subprocess.check_output(shlex.split(cmd)).decode().strip()
            print('[GPU]', out)
        except Exception as e:
            print('[GPU monitor error]', e)
            break
        time.sleep(interval)

mon_thread = threading.Thread(target=gpu_monitor, args=(30,), daemon=True)
mon_thread.start()

## 4) Google Colab: Drive Bağlantısı ve 7z/NPY İşleme

In [ ]:
# Drive montajı ve dosya çıkarma yardımcıları
import shutil, pathlib, py7zr

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

# Örnek: Drive'da saklanan arşiv ve npy dizinleri
DRIVE_BASE = '/content/drive/MyDrive/wakeword'
ARCHIVES = [
    # ('source_path_in_drive', 'target_dir_name_under_content')
    (f'{DRIVE_BASE}/positive_dataset.7z', 'positive_dataset'),
    (f'{DRIVE_BASE}/negative_dataset.7z', 'negative_dataset'),
]
NPY_SOURCES = [
    (f'{DRIVE_BASE}/features_train_npy', '/content/features/train'),
]

runtime_base = '/content' if IN_COLAB else os.getcwd()

# Arşivleri kopyala ve çıkar
def extract_archives(archives):
    for src, target_name in archives:
        target_dir = os.path.join(runtime_base, target_name)
        os.makedirs(target_dir, exist_ok=True)
        local_7z = os.path.join(runtime_base, os.path.basename(src))
        if os.path.exists(src):
            print('Kopyalanıyor:', src, '->', local_7z)
            shutil.copy2(src, local_7z)
            print('Çıkarılıyor:', local_7z, '->', target_dir)
            with py7zr.SevenZipFile(local_7z, mode='r') as z:
                z.extractall(path=target_dir)
        else:
            print('Bulunamadı (atlandı):', src)

# NPY dizinlerini kopyala
def copy_npy_dirs(pairs):
    for src_dir, dst_dir in pairs:
        if os.path.exists(src_dir):
            os.makedirs(dst_dir, exist_ok=True)
            print('NPY kopyalanıyor:', src_dir, '->', dst_dir)
            # Recurse copy
            for root, dirs, files in os.walk(src_dir):
                rel = os.path.relpath(root, src_dir)
                out_root = os.path.join(dst_dir, rel)
                os.makedirs(out_root, exist_ok=True)
                for f in files:
                    if f.endswith('.npy'):
                        shutil.copy2(os.path.join(root, f), os.path.join(out_root, f))
        else:
            print('NPY kaynak yok (atlandı):', src_dir)

if IN_COLAB:
    extract_archives(ARCHIVES)
    copy_npy_dirs(NPY_SOURCES)

## 5) Yol Yapılandırması (Local ve Colab)

In [ ]:
# Yol ve konfig
from pathlib import Path
IN_COLAB = 'google.colab' in sys.modules
BASE = Path('/content') if IN_COLAB else Path(os.getcwd())
DATA_POS = str(BASE / 'positive_dataset')
DATA_NEG = str(BASE / 'negative_dataset')
FEATURES_DIR = str(BASE / 'features' / 'train')
CKPT_DIR = str(BASE / 'checkpoints'); Path(CKPT_DIR).mkdir(parents=True, exist_ok=True)
RUNS_DIR = str(BASE / 'runs'); Path(RUNS_DIR).mkdir(parents=True, exist_ok=True)

print('POS:', DATA_POS)
print('NEG:', DATA_NEG)
print('FEAT:', FEATURES_DIR)
print('CKPT:', CKPT_DIR)
print('RUNS:', RUNS_DIR)

## 6) Veri Yükleyicileri (Dataset/DataLoader)

In [ ]:
# Proje dataset ve dataloader
sys.path.append(os.getcwd())
from training.enhanced_dataset import EnhancedAudioConfig, create_dataloaders

audio_cfg = EnhancedAudioConfig(
    use_precomputed_features=True,
    features_dir=FEATURES_DIR,
)
train_loader, val_loader = create_dataloaders(
    positive_dir=DATA_POS,
    negative_dir=DATA_NEG,
    features_dir=FEATURES_DIR,
    batch_size=32,
    config=audio_cfg,
)
print('Train/Val:', len(train_loader), len(val_loader))

## 7) Model Tanımı (PyTorch)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ModelCfg:
    input_size = 40
    hidden_size = 256
    num_layers = 2
    dropout = 0.6
    num_classes = 2

class WakeModel(nn.Module):
    def __init__(self, cfg: ModelCfg):
        super().__init__()
        self.cfg = cfg
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout_cnn = nn.Dropout(0.3)
        self.lstm = nn.LSTM(
            input_size=128 * (cfg.input_size // 8),
            hidden_size=cfg.hidden_size,
            num_layers=cfg.num_layers,
            batch_first=True,
            dropout=cfg.dropout if cfg.num_layers>1 else 0,
            bidirectional=True,
        )
        self.attn = nn.Linear(cfg.hidden_size*2, 1)
        self.drop = nn.Dropout(cfg.dropout)
        self.fc1 = nn.Linear(cfg.hidden_size*2, 128)
        self.fc2 = nn.Linear(128, cfg.num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout_cnn(x)
        B,C,Hp,Wp = x.shape
        x = x.permute(0,3,1,2).contiguous().view(B, Wp, C*Hp)
        lstm_out,_ = self.lstm(x)
        a = torch.softmax(self.attn(lstm_out), dim=1)
        z = torch.sum(a*lstm_out, dim=1)
        z = self.drop(z)
        z = F.relu(self.fc1(z))
        return self.fc2(z)

model_cfg = ModelCfg()
model = WakeModel(model_cfg).to(device)
print('Model params:', sum(p.numel() for p in model.parameters()))

## 8) Kayıp, Optimizasyon ve TorchMetrics

In [ ]:
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryF1Score
from torch.utils.tensorboard import SummaryWriter

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5)

acc_tr = BinaryAccuracy().to(device)
pre_tr = BinaryPrecision().to(device)
rec_tr = BinaryRecall().to(device)
f1_tr  = BinaryF1Score().to(device)

acc_va = BinaryAccuracy().to(device)
pre_va = BinaryPrecision().to(device)
rec_va = BinaryRecall().to(device)
f1_va  = BinaryF1Score().to(device)

from datetime import datetime
run_name = datetime.now().strftime('%Y%m%d-%H%M%S')
writer = SummaryWriter(log_dir=os.path.join(RUNS_DIR, f'colab-{run_name}'))
print('TB dir:', writer.log_dir)

## 9) TensorBoard Kurulumu ve Yazar

In [ ]:
# Hiperparametreleri TB'a yaz
writer.add_text('hparams', str({
    'batch_size': 32,
    'lr': 1e-4,
    'scheduler': 'ReduceLROnPlateau(0.5, patience=5)',
    'save_every': 10,
}))

# Opsiyonel: Model grafiği için dummy input
# Not: giriş boyutunuz (B,C,T) = (1,40,?) ise aşağıdaki örnek çalışır hale getirilebilir
# try:
#     dummy = torch.randn(1, 40, 128).to(device)
#     writer.add_graph(model, dummy)
# except Exception as e:
#     print('Graph eklenemedi:', e)

## 10) Eğitim Döngüsü (Checkpoint her 10 epoch + TensorBoard + Metrics)

In [ ]:
from pathlib import Path

ckpt_dir = Path(CKPT_DIR)
last_ckpt = ckpt_dir / 'last_checkpoint.pth'
best_ckpt = ckpt_dir / 'best_model.pth'

start_epoch = 0
best_val_f1 = -1.0

# Eğitim yardımcıları
def reset_train_metrics():
    acc_tr.reset(); pre_tr.reset(); rec_tr.reset(); f1_tr.reset()

def reset_val_metrics():
    acc_va.reset(); pre_va.reset(); rec_va.reset(); f1_va.reset()

def evaluate():
    model.eval()
    reset_val_metrics()
    total = 0.0
    with torch.no_grad():
        for batch in val_loader:
            x = batch['features'].to(device)
            y = batch['label'].to(device)
            logits = model(x)
            loss = criterion(logits, y)
            total += loss.item()
            probs = torch.softmax(logits, dim=1)[:,1]
            acc_va.update(probs, y)
            pre_va.update(probs, y)
            rec_va.update(probs, y)
            f1_va.update(probs, y)
    n = max(1, len(val_loader))
    return {
        'loss': total / n,
        'acc': acc_va.compute().item(),
        'prec': pre_va.compute().item(),
        'rec': rec_va.compute().item(),
        'f1': f1_va.compute().item(),
    }

def save_ckpt(epoch, tag=None):
    tag_suffix = f"_epoch_{epoch:04d}" if tag is None else f"_{tag}"
    path = ckpt_dir / f"checkpoint{tag_suffix}.pth"
    torch.save({
        'epoch': epoch,
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
        'scheduler_state': scheduler.state_dict() if scheduler else None,
        'best_val_f1': best_val_f1,
    }, path)
    torch.save({
        'epoch': epoch,
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
        'scheduler_state': scheduler.state_dict() if scheduler else None,
        'best_val_f1': best_val_f1,
    }, last_ckpt)
    print('Saved:', path)

EPOCHS = 50

for epoch in range(start_epoch, EPOCHS):
    model.train()
    reset_train_metrics()
    running = 0.0
    for batch in train_loader:
        x = batch['features'].to(device)
        y = batch['label'].to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        running += loss.item()
        probs = torch.softmax(logits, dim=1)[:,1]
        acc_tr.update(probs, y)
        pre_tr.update(probs, y)
        rec_tr.update(probs, y)
        f1_tr.update(probs, y)

    tr_loss = running / max(1, len(train_loader))
    tr_acc, tr_pre, tr_rec, tr_f1 = [m.item() for m in (acc_tr.compute(), pre_tr.compute(), rec_tr.compute(), f1_tr.compute())]

    val_stats = evaluate()
    if scheduler:
        scheduler.step(val_stats['loss'])

    # TB yaz
    writer.add_scalar('Loss/train', tr_loss, epoch)
    writer.add_scalar('Loss/val', val_stats['loss'], epoch)
    writer.add_scalar('Acc/train', tr_acc, epoch)
    writer.add_scalar('Acc/val', val_stats['acc'], epoch)
    writer.add_scalar('F1/train', tr_f1, epoch)
    writer.add_scalar('F1/val', val_stats['f1'], epoch)
    writer.add_scalar('LR', optimizer.param_groups[0]['lr'], epoch)

    print(f"Epoch {epoch+1}/{EPOCHS} | tr_loss={tr_loss:.4f} val_loss={val_stats['loss']:.4f} tr_f1={tr_f1:.4f} val_f1={val_stats['f1']:.4f}")

    # En iyi model kaydı
    if val_stats['f1'] > best_val_f1:
        best_val_f1 = val_stats['f1']
        torch.save({
            'epoch': epoch,
            'model_state': model.state_dict(),
            'optimizer_state': optimizer.state_dict(),
            'scheduler_state': scheduler.state_dict() if scheduler else None,
            'best_val_f1': best_val_f1,
        }, best_ckpt)
        print('Best model updated')

    # 10 epoch'ta bir checkpoint
    if (epoch + 1) % 10 == 0:
        save_ckpt(epoch+1)

# Final
save_ckpt(EPOCHS, tag='final')
writer.close()

gpu_monitor_stop = True

## 11) Doğrulama/Test Döngüsü
(Üstte evaluate() fonksiyonu tanımlandı.)

## 12) Checkpoint Yükleme ve Eğitime Devam

In [ ]:
# Örnek: checkpoint yükleme
ckpt_path = str(Path(CKPT_DIR) / 'last_checkpoint.pth')
if os.path.exists(ckpt_path):
    state = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(state['model_state'])
    optimizer.load_state_dict(state['optimizer_state'])
    if state.get('scheduler_state') and scheduler:
        scheduler.load_state_dict(state['scheduler_state'])
    print('Checkpoint yüklendi. Kaldığı yerden devam edebilirsiniz.')
else:
    print('Checkpoint bulunamadı. Yeni eğitim başlatın.')

## 13) Colab: TensorBoard Çalıştırma

In [ ]:
# Colab içinde TB
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    %load_ext tensorboard
    %tensorboard --logdir $RUNS_DIR --host 0.0.0.0 --port 6006
else:
    print('Lokal JupyterLab kullanıyorsanız terminalde çalıştırın:')
    print('tensorboard --logdir runs --host 127.0.0.1 --port 6006')

## 14) Çalıştırma Konfigürasyonu ve CLI Argümanları

In [ ]:
# Basit konfig sözlüğü
config = {
    'batch_size': 32,
    'lr': 1e-4,
    'epochs': 50,
    'save_every': 10,
    'num_workers': 0,
    'pin_memory': torch.cuda.is_available(),
    'pos_dir': DATA_POS,
    'neg_dir': DATA_NEG,
    'features_dir': FEATURES_DIR,
    'ckpt_dir': CKPT_DIR,
    'runs_dir': RUNS_DIR,
}
print(config)

## 15) GPU Kullanımı Canlı İzleme İş Parçacığı
(Yukarıda başlatıldı; eğitim sonunda durduruldu.)